# Resume Data - Entity Extraction and Graph Loading

In [1]:
from dotenv import load_dotenv
import os
load_dotenv('.streamlit/secrets.toml', override=True)

# Neo4j
NEO4J_URI = os.getenv('RESUME_NEO4J_URI')
NEO4J_USERNAME = os.getenv('RESUME_NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('RESUME_NEO4J_PASSWORD')

#OPENAI
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## GenAI Setup

In [2]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(temperature=0, model_name="gpt-4-0125-preview")

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm for resumes from job aspirants. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return an empty string, '', for the attribute's value."
            "Do not create fictitious data or impute missing values."
        ),
        ("human", "{text}"),
    ]
)

In [4]:
import datetime
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field

class Position(BaseModel):
    id: str = Field(description="Unique position id")
    title: str = Field(description="The job title")
    location: str = Field(description="Location of position")
    startDate: str = Field(description="Start date of position")
    endDate: str = Field(description="End date of position")
    description: str = Field(description="A crisp text summary of position that MUST NOT be more than 100 characters")
    company: str = Field(description="Name of company they worked the position for")

class Skill(BaseModel):
    id: str = Field(description="Unique skill id")
    name: str = Field(description="The name of the skill")
    level: str = Field(description="Experience level")

class Education(BaseModel):
    id: str = Field(description="Unique education id")
    degree: str = Field(description="Name of educational degree")
    institution: str = Field(description="Name of educational institution")
    location: str = Field(description="Location of educational institution")
    graduationDate: str = Field(description="Date of graduation")

class Person(BaseModel):
    id: str = Field(description="Unique person id")
    role: str = Field(description="The job/employment role")
    description: str = Field(description="A crisp text summary and MUST NOT be more than 250 characters")
    positions: List[Position]
    skills: List[Skill]
    education: List[Education]


In [5]:
chain = prompt | llm.with_structured_output(Person)

/Users/zachblumenfeld/opt/anaconda3/envs/genai-workshop/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


## Neo4j Setup

In [6]:
from langchain_community.graphs.neo4j_graph import Neo4jGraph

graph = Neo4jGraph(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
graph.query('CREATE CONSTRAINT person_entityId IF NOT EXISTS FOR (p:Person) REQUIRE (p.entityId) IS UNIQUE;')

graph.query('CREATE CONSTRAINT position_entityId IF NOT EXISTS FOR (p:Position) REQUIRE (p.entityId) IS UNIQUE;')
graph.query('CREATE CONSTRAINT company_entityId IF NOT EXISTS FOR (p:Company) REQUIRE (p.entityId) IS UNIQUE;')
graph.query('CREATE CONSTRAINT job_title_entityId IF NOT EXISTS FOR (p:JobTitle) REQUIRE (p.entityId) IS UNIQUE;')

graph.query('CREATE CONSTRAINT skill_entityId IF NOT EXISTS FOR (p:Skill) REQUIRE (p.entityId) IS UNIQUE;')

graph.query('CREATE CONSTRAINT education_entityId IF NOT EXISTS FOR (p:Education) REQUIRE (p.entityId) IS UNIQUE;')

[]

## Extraction & Loading

In [16]:
TOTAL_DOCUMENTS = 200
LOAD_CHUNK_SIZE = 20

In [17]:
import glob
import re

def clean_text(t):
    return re.sub(r'[^\x00-\x7F]+',' ', t)

def chunks(xs, n: int = 1_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]


In [18]:
def format_list_dict(objs: BaseModel, source_id):
    res = []
    for obj in objs:
        d = obj.dict()
        d['sourceId'] =  source_id
        res.append(d)
    return res



def extract_data(txt_files):
    people = []
    positions = []
    skills = []
    educations = []
    failed_files = []

    for i in range(len(txt_files)):
        with open(txt_files[i], 'r', encoding='utf-8', errors='ignore') as file:
            text = clean_text(file.read().rstrip())
            try:
                person = chain.invoke(text)
                people.append({
                    'id':person.id,
                    'role': person.role,
                    'description': person.description,
                    'sourceId': txt_files[i]
                })
                positions.extend(format_list_dict(person.positions, txt_files[i]))
                skills.extend(format_list_dict(person.skills, txt_files[i]))
                educations.extend(format_list_dict(person.education, txt_files[i]))
                print(f"Successfully processed {i+1} of {len(txt_files)}")
            except Exception as e:
                print(f"{txt_files[i]}: Processing Failed with exception {e}")
                failed_files.append(txt_files[i])
    return people, positions, skills, educations, failed_files

def load_data(people, positions, skills, educations):
    for recs in chunks(people):
        graph.query('''
        UNWIND $recs AS rec
        MERGE(n:Person {entityId: rec.sourceId})
        SET n += rec
        RETURN count(n)
        ''', params={'recs': recs})

    for recs in chunks(positions):
        graph.query('''
        UNWIND $recs AS rec
        MATCH(p:Person {entityId: rec.sourceId})
        MERGE(n:Position {entityId: rec.sourceId + ' - ' + rec.id})
        SET n += rec
        MERGE(p)-[:HAS_POSITION]->(n)
        WITH n
        MERGE(j:JobTitle {entityId: toUpper(n.title)})
        MERGE(n)-[r:WITH_TITLE]->(j)
        WITH n
        WHERE n.company <> ""
        MERGE(c:Company {entityId: toUpper(n.company)})
        MERGE(n)-[r:AT_COMPANY]->(c)
        RETURN count(n)
        ''', params={'recs': recs})
    for recs in chunks(skills):
        graph.query('''
        UNWIND $recs AS rec
        MATCH(p:Person {entityId: rec.sourceId})
        MERGE(n:Skill {entityId: toUpper(rec.name)})
        MERGE(p)-[r:HAS_SKILL]->(n)
        SET r += rec
        RETURN count(r)
        ''', params={'recs': recs})
    for recs in chunks(educations):
        graph.query('''
        UNWIND $recs AS rec
        MATCH(p:Person {entityId: rec.sourceId})
        MERGE(n:Education {entityId: rec.sourceId + ' - ' + rec.id})
        MERGE(p)-[:HAS_EDUCATION]->(n)
        SET n += rec
        RETURN count(n)
        ''', params={'recs': recs})

In [19]:
%%time

from tqdm import tqdm

text_files = glob.glob("data/*.txt")[:TOTAL_DOCUMENTS]
failed_files_list = []

for txt_file_seg in tqdm(chunks(text_files, LOAD_CHUNK_SIZE)):
    print('======= Extracting Data From Files Segment ========')
    people, positions, skills, educations, failed_files = extract_data(txt_file_seg)
    print('Completed Extraction From Files Segment')
    failed_files_list.extend(failed_files)
    print('======= Loading Extracted Data ========')
    load_data(people, positions, skills, educations)


  0%|          | 0/10 [00:00<?, ?it/s]

======= Extracting Data From Files Segment ========
Successfully processed 1 of 20
Successfully processed 2 of 20
Successfully processed 3 of 20
Successfully processed 4 of 20
Successfully processed 5 of 20
Successfully processed 6 of 20
Successfully processed 7 of 20
data/09558.txt: Processing Failed with exception 1 validation error for Person
education -> 1 -> location
  field required (type=value_error.missing)
Successfully processed 9 of 20
Successfully processed 10 of 20
Successfully processed 11 of 20
Successfully processed 12 of 20
Successfully processed 13 of 20
Successfully processed 14 of 20
Successfully processed 15 of 20
Successfully processed 16 of 20
Successfully processed 17 of 20
Successfully processed 18 of 20
Successfully processed 19 of 20
Successfully processed 20 of 20
Completed Extraction From Files Segment
======= Loading Extracted Data ========


 10%|█         | 1/10 [07:54<1:11:11, 474.59s/it]

======= Extracting Data From Files Segment ========
Successfully processed 1 of 20
Successfully processed 2 of 20
Successfully processed 3 of 20
Successfully processed 4 of 20
Successfully processed 5 of 20
Successfully processed 6 of 20
Successfully processed 7 of 20
Successfully processed 8 of 20
Successfully processed 9 of 20
Successfully processed 10 of 20
Successfully processed 11 of 20
Successfully processed 12 of 20
Successfully processed 13 of 20
Successfully processed 14 of 20
Successfully processed 15 of 20
Successfully processed 16 of 20
Successfully processed 17 of 20
Successfully processed 18 of 20
Successfully processed 19 of 20
Successfully processed 20 of 20
Completed Extraction From Files Segment
======= Loading Extracted Data ========


 20%|██        | 2/10 [14:40<57:51, 433.91s/it]  

...

======= Extracting Data From Files Segment ========
Successfully processed 1 of 20
Successfully processed 2 of 20
Successfully processed 3 of 20
Successfully processed 4 of 20
Successfully processed 5 of 20
Successfully processed 6 of 20
Successfully processed 7 of 20
Successfully processed 8 of 20
Successfully processed 9 of 20
Successfully processed 10 of 20
Successfully processed 11 of 20
Successfully processed 12 of 20
Successfully processed 13 of 20
Successfully processed 14 of 20
Successfully processed 15 of 20
Successfully processed 16 of 20
Successfully processed 17 of 20
Successfully processed 18 of 20
Successfully processed 19 of 20
Successfully processed 20 of 20
Completed Extraction From Files Segment
======= Loading Extracted Data ========


 90%|█████████ | 9/10 [1:07:59<07:35, 455.70s/it]

======= Extracting Data From Files Segment ========
Successfully processed 1 of 20
Successfully processed 2 of 20
Successfully processed 3 of 20
Successfully processed 4 of 20
Successfully processed 5 of 20
Successfully processed 6 of 20
Successfully processed 7 of 20
Successfully processed 8 of 20
Successfully processed 9 of 20
Successfully processed 10 of 20
Successfully processed 11 of 20
Successfully processed 12 of 20
Successfully processed 13 of 20
Successfully processed 14 of 20
Successfully processed 15 of 20
Successfully processed 16 of 20
Successfully processed 17 of 20
Successfully processed 18 of 20
Successfully processed 19 of 20
Successfully processed 20 of 20
Completed Extraction From Files Segment
======= Loading Extracted Data ========


100%|██████████| 10/10 [1:14:59<00:00, 449.91s/it]

CPU times: user 3.89 s, sys: 574 ms, total: 4.46 s
Wall time: 1h 14min 59s


## Generate Representative Names

In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

name_prompt_tmpl = """
Provide a likely name for a person based on the below location info for past education and positions.
Just provide the name itself without any additional explanation or text.
If you aren't sure of a good name make one up.

{info}
"""
name_prompt = PromptTemplate.from_template(name_prompt_tmpl)
name_llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
name_chain = name_prompt | name_llm | StrOutputParser()

In [45]:
persons_info = graph.query('''
MATCH(p:Person)
WITH p
OPTIONAL MATCH(p)-[:HAS_POSITION]->(s)
WITH p, collect({title:s.title, timePeriod:s.startDate + " - " + s.endDate, location:s.location}) AS positions
OPTIONAL MATCH(p)-[:HAS_EDUCATION]->(e)
RETURN p.entityId AS entityId, positions, collect({degree:e.degree, graduationDatee:e.graduationDate,
    institution:e.institution, location:e.location}) AS education
''')
persons_info[:3]

[{'entityId': 'data/08652.txt',
  'positions': [{'timePeriod': 'July 2012 - Present',
    'title': 'Sales/ Project Manager/ IT Professional',
    'location': 'Little Rock, AR'}],
  'education': [{'degree': 'Bachelors in Management Information Systems',
    'location': 'Little Rock, AR',
    'graduationDatee': '2013',
    'institution': 'University of Arkansas at Little Rock'},
   {'degree': 'Bachelors in Accounting',
    'location': 'Little Rock, AR',
    'graduationDatee': '2013',
    'institution': 'University of Arkansas at Little Rock'},
   {'degree': 'Associates of Arts in General',
    'location': '',
    'graduationDatee': 'March 2010',
    'institution': 'Pulaski Technical College'}]},
 {'entityId': 'data/07561.txt',
  'positions': [{'timePeriod': 'November 2018 - ',
    'title': 'Project Manager',
    'location': 'Bloomfield, NJ'},
   {'timePeriod': 'December 2016 - November 2018',
    'title': 'IT Support Coordinator',
    'location': 'Bloomfield, NJ'},
   {'timePeriod': 'Mar

In [46]:
len(persons_info)

199

In [48]:
import json

name_records = []
for person_info in tqdm(persons_info):
    res = name_chain.invoke(json.dumps(person_info, indent=1))
    name_records.append({'entityId':person_info['entityId'], 'name':res})

100%|██████████| 199/199 [01:36<00:00,  2.06it/s]


In [49]:
name_records[10:]

[{'entityId': 'data/06643.txt', 'name': 'Samantha Johnson'},
 {'entityId': 'data/09570.txt', 'name': 'Ryan Johnson'},
 {'entityId': 'data/10021.txt', 'name': 'Tyler Johnson'},
 {'entityId': 'data/08108.txt', 'name': 'Ryan Johnson'},
 {'entityId': 'data/06125.txt', 'name': 'Nathan Reynolds'},
 {'entityId': 'data/09216.txt', 'name': 'Nathan Patel'},
 {'entityId': 'data/06131.txt', 'name': 'Natalie Thompson'},
 {'entityId': 'data/09202.txt', 'name': 'Ryan Johnson'},
 {'entityId': 'data/06657.txt', 'name': 'Samantha Rodriguez'},
 {'entityId': 'data/09564.txt', 'name': 'Nathan Washington'},
 {'entityId': 'data/07549.txt', 'name': 'Ryan Sullivan'},
 {'entityId': 'data/10035.txt', 'name': 'Olufemi Adeyemi'},
 {'entityId': 'data/08691.txt', 'name': 'Emily Johnson'},
 {'entityId': 'data/06864.txt', 'name': 'Alexandra Smith'},
 {'entityId': 'data/08849.txt', 'name': 'Nathan Johnson'},
 {'entityId': 'data/08685.txt', 'name': 'Tyler Johnson'},
 {'entityId': 'data/06870.txt', 'name': 'Nkechi Adebow

In [50]:
graph.query('''
UNWIND $recs AS rec
MATCH(n:Person {entityId: rec.entityId})
SET n.name = rec.name
RETURN count(n)
''', params={'recs': name_records})

[{'count(n)': 199}]

In [51]:
persons_info = graph.query('''
MATCH(p:Person)
RETURN p.entityId AS entityId, p.name AS name, p.role AS role, p.description AS description
''')
persons_info[:3]

[{'entityId': 'data/08652.txt',
  'name': 'Tyler Johnson',
  'role': 'Sales/ Project Manager/ IT Professional',
  'description': 'Experienced in sales, project management, and IT system improvements. Skilled in database development, QAC analysis, and custom software creation.'},
 {'entityId': 'data/07561.txt',
  'name': 'Samantha Rodriguez',
  'role': 'Project Manager',
  'description': 'Seeking a career opportunity that will allow me to utilize my administrative, communication, and problem solving skills.'},
 {'entityId': 'data/08134.txt',
  'name': 'Natalie Rodriguez',
  'role': 'Operations Analyst',
  'description': 'Experienced professional with 18 years supporting and delivering solutions across global operations. Skilled in proposal and contract management, project coordination, and leading cross-functional teams.'}]

In [65]:
img_prompt_tmpl = '''
Generate an profile image of this person based on the below data:
'''

print(img_prompt_tmpl + json.dumps(persons_info[0], indent=1))


Generate an profile image of this person based on the below data:
{
 "entityId": "data/08652.txt",
 "name": "Tyler Johnson",
 "role": "Sales/ Project Manager/ IT Professional",
 "description": "Experienced in sales, project management, and IT system improvements. Skilled in database development, QAC analysis, and custom software creation."
}


In [77]:
if not os.path.exists('img/data'):
    os.makedirs('img/data')

In [81]:
import requests
from openai import OpenAI
client = OpenAI()

for person_info in tqdm(persons_info[135:]):
    try:
        response = client.images.generate(
            model="dall-e-2",
            prompt=img_prompt_tmpl + json.dumps(person_info, indent=1),
            size="256x256",
            quality="standard",
            response_format= "url", #"b64_json",
            n=1,
        )

        img_data = requests.get(response.data[0].url).content
        with open(f'img/{person_info["entityId"][:-4]}.jpg', 'wb') as handler:
            handler.write(img_data)
    except Exception as e:
        print(f"{person_info['entityId']}: Processing Failed with exception {e}")

  3%|▎         | 2/64 [00:13<05:39,  5.48s/it]

data/09404.txt: Processing Failed with exception Error code: 400 - {'error': {'code': 'content_policy_violation', 'message': 'Your request was rejected as a result of our safety system. Your prompt may contain text that is not allowed by our safety system.', 'param': None, 'type': 'invalid_request_error'}}


100%|██████████| 64/64 [09:59<00:00,  9.37s/it]


## Create Vector Properties & Indexes

In [85]:
graph.query('''
MATCH (n:Person) WHERE size(n.description) <> 0
WITH collect(n) AS nodes, toInteger(rand()*$numberOfBatches) AS partition
CALL {
    WITH nodes
    CALL genai.vector.encodeBatch([node IN nodes| node.description], "OpenAI", { token: $token})
    YIELD index, vector
    CALL db.create.setNodeVectorProperty(nodes[index], "textEmbedding", vector)
} IN TRANSACTIONS OF 1 ROW
''', params={'token':OPENAI_API_KEY, 'numberOfBatches':8})

[]

In [89]:
embedding_dimension = 1536
graph.query('''
CREATE VECTOR INDEX person_text_embedding IF NOT EXISTS FOR (n:Person) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dim),
 `vector.similarity_function`: 'cosine'
}}''', params={'dim': embedding_dimension})

graph.query('CALL db.awaitIndex("person_text_embedding", 300)')

[]

In [86]:
graph.query('''
MATCH (n:Skill) WHERE size(n.entityId) <> 0
WITH collect(n) AS nodes, toInteger(rand()*$numberOfBatches) AS partition
CALL {
    WITH nodes
    CALL genai.vector.encodeBatch([node IN nodes| node.entityId], "OpenAI", { token: $token})
    YIELD index, vector
    CALL db.create.setNodeVectorProperty(nodes[index], "textEmbedding", vector)
} IN TRANSACTIONS OF 1 ROW
''', params={'token':OPENAI_API_KEY, 'numberOfBatches':40})

[]

In [90]:
graph.query('''
CREATE VECTOR INDEX skill_text_embedding IF NOT EXISTS FOR (n:Skill) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dim),
 `vector.similarity_function`: 'cosine'
}}''', params={'dim': embedding_dimension})

graph.query('CALL db.awaitIndex("skill_text_embedding", 300)')

[]

In [87]:
graph.query('''
MATCH (n:Position) WHERE size(n.description) <> 0
WITH collect(n) AS nodes, toInteger(rand()*$numberOfBatches) AS partition
CALL {
    WITH nodes
    CALL genai.vector.encodeBatch([node IN nodes| node.description], "OpenAI", { token: $token})
    YIELD index, vector
    CALL db.create.setNodeVectorProperty(nodes[index], "textEmbedding", vector)
} IN TRANSACTIONS OF 1 ROW
''', params={'token':OPENAI_API_KEY, 'numberOfBatches':40})

[]

In [91]:
graph.query('''
CREATE VECTOR INDEX position_text_embedding IF NOT EXISTS FOR (n:Position) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dim),
 `vector.similarity_function`: 'cosine'
}}''', params={'dim': embedding_dimension})

graph.query('CALL db.awaitIndex("position_text_embedding", 300)')

[]

## Create Full Text Embeddings

In [93]:
graph.query('''
CREATE FULLTEXT INDEX person_full_text IF NOT EXISTS FOR (n:Person) ON EACH [n.role]
''')

graph.query('CALL db.awaitIndex("person_full_text", 300)')

[]

In [94]:
graph.query('''
CREATE FULLTEXT INDEX skill_full_text IF NOT EXISTS FOR (n:Skill) ON EACH [n.entityId]
''')

graph.query('CALL db.awaitIndex("skill_full_text", 300)')

[]

In [95]:
graph.query('''
CREATE FULLTEXT INDEX position_full_text IF NOT EXISTS FOR (n:Position) ON EACH [n.title]
''')

graph.query('CALL db.awaitIndex("position_full_text", 300)')

[]